In [ ]:
import os
import numpy as np
import pandas as pd
from flask import Flask, request
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
from collections.abc import Iterable
import pickle
from flask_cors import CORS
import math

app = Flask(__name__)
CORS(app)

@app.route('/')
def hello():
    return "Hello An!"


def weightedRating(x):
    v = x['Rating']
    R = x['AggregatedRating']
    return (v / (v + 51.0) * R) + (51.0 / (51.0 + v) * 4.56)

categoryBasedRecommender = pickle.load(open('categoryBasedRecommender.pkl', 'rb'))
metadataBasedRecommender = pickle.load(open('metadataBasedRecommender.pkl', 'rb'))

datasetDumpAlgolia = pickle.load(open('datasetDump.pkl', 'rb'))


# weighted popularity model
#qualifiedRecipes = pickle.load(open('qualifiedRecipes.pkl', 'rb'))


qualifiedRecipes = pickle.load(open('qualifiedRecipesCollaborative.pkl', 'rb'))
qualifiedRecipes['score'] = qualifiedRecipes.apply(weightedRating, axis=1)
df_rating = pd.DataFrame(qualifiedRecipes.groupby(by = ['RecipeId'])['score'].agg('mean')) 

filtered_ratings = pickle.load(open('filtered_ratings.pkl', 'rb'))
collaborativeMatrix = filtered_ratings.pivot(index = 'AuthorId', columns ='RecipeId', values = 'Rating').fillna(0)

recipe_metadata = qualifiedRecipes[['RecipeId','Name','RecipeCategory']]
recipe_data = filtered_ratings.merge(recipe_metadata,on='RecipeId')
knnMatrix = recipe_data.pivot(index = 'RecipeId', columns ='AuthorId', values = 'Rating').fillna(0)
    

@app.route('/popularity')
def weightedRatingPopularity():
    return qualifiedRecipes.to_json()


# category - got the data in weightedPopularRecipes and now creating the route function
@app.route('/category', methods=['POST'])
def getCategoryWiseRecommendations():
    data = request.get_json()
    RecipeCategory = data['keyword']
    percentile = 0.85
    categorySpecificRecipes = categoryBasedRecommender[categoryBasedRecommender['Category'] == RecipeCategory]
    RatingCounts = categorySpecificRecipes[categorySpecificRecipes['Rating'].notnull()]['Rating'].astype('int')
    RatingAverages = categorySpecificRecipes[categorySpecificRecipes['AggregatedRating'].notnull()][
        'AggregatedRating'].astype('int')
    C = RatingAverages.mean()
    m = RatingCounts.quantile(percentile)

    qualified = categorySpecificRecipes[
        (categorySpecificRecipes['Rating'] >= m) & (categorySpecificRecipes['Rating'].notnull()) & (
            categorySpecificRecipes['AggregatedRating'].notnull())][
        ['RecipeId', 'Name', 'Rating', 'AggregatedRating', 'ReviewCount']]
    qualified['Rating'] = qualified['Rating'].astype('int')
    qualified['AggregatedRating'] = qualified['AggregatedRating'].astype('int')

    qualified['wr'] = qualified.apply(
        lambda x: (x['Rating'] / (x['Rating'] + m) * x['AggregatedRating']) + (m / (m + x['Rating']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(12)

    return qualified.to_json()


# metadata
termFrequency2 = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0.0, stop_words='english')
tfidMatrix2 = termFrequency2.fit_transform(metadataBasedRecommender['Metadata'])
cosineSimilarity2 = linear_kernel(tfidMatrix2, tfidMatrix2)

metadataBasedRecommender = metadataBasedRecommender.reset_index()
names2 = metadataBasedRecommender[['RecipeId', 'Name']]
indices2 = pd.Series(metadataBasedRecommender.index, index=metadataBasedRecommender['Name'])


@app.route('/metadata', methods=['POST'])
def getMetadataBasedRecommendations():
    data = request.get_json()
    title = data['keyword']
    idx = indices2[title]
    if isinstance(idx, Iterable):
        for i in idx:
            similarityScores = sorted(list(enumerate(cosineSimilarity2[i])), key=lambda x: x[1], reverse=True)[1:13]
            break
    else:
        similarityScores = sorted(list(enumerate(cosineSimilarity2[idx])), key=lambda x: x[1], reverse=True)[1:13]
    recipeIndices = [i[0] for i in similarityScores]
    return names2.iloc[recipeIndices].to_json()


# content with ratings considered
contentBasedRecommender = metadataBasedRecommender


@app.route('/content', methods=['POST'])
def getContentBasedRecommendations():
    data = request.get_json()
    title = data['keyword']
    idx = indices2[title]

    if isinstance(idx, Iterable):
        recipeDump = pd.DataFrame()
        for i in idx:
            cnt = 0
            similarityScores = sorted(list(enumerate(cosineSimilarity2[i])), key=lambda x: x[1], reverse=True)[1:26]
            recipeIndices = [i[0] for i in similarityScores]
            recipeSimilarity = [i[1] for i in similarityScores]
            
            contentBasedRecommendedRecipes = contentBasedRecommender.iloc[recipeIndices][
                ['RecipeId', 'Name', 'Rating', 'AggregatedRating']]
            ratingCounts = contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['Rating'].notnull()][
                'Rating'].astype('int')
            ratingAverages = \
            contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['AggregatedRating'].notnull()][
                'AggregatedRating'].astype('int')
            C = ratingAverages.mean()
            m = ratingCounts.quantile(0.60)
            contentBasedRecommendedRecipes['Similarity'] = recipeSimilarity
            qualifiedRecipes2 = contentBasedRecommendedRecipes[(contentBasedRecommendedRecipes['Rating'] >= m) & (
                contentBasedRecommendedRecipes['Rating'].notnull()) & (contentBasedRecommendedRecipes[
                                                                           'AggregatedRating'].notnull())]
            qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
            qualifiedRecipes2['AggregatedRating'] = qualifiedRecipes2['AggregatedRating'].astype('int').head(12)
            if cnt == 0:
                recipeDump = qualifiedRecipes2
            else:
                recipeDump.append(qualifiedRecipes2, ignore_index=True)
            cnt += 1
        recipeDump = recipeDump.sort_values('Similarity', ascending=False)
        recipeDump = recipeDump.reset_index()
        return recipeDump.head(12).to_json()
 
    else:
        similarityScores = sorted(list(enumerate(cosineSimilarity2[idx])), key=lambda x: x[1], reverse=True)[1:26]
        recipeIndices = [i[0] for i in similarityScores]
        recipeSimilarity = [i[1] for i in similarityScores]
        
        contentBasedRecommendedRecipes = contentBasedRecommender.iloc[recipeIndices][
            ['RecipeId', 'Name', 'Rating', 'AggregatedRating']]
        ratingCounts = contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['Rating'].notnull()][
            'Rating'].astype('int')
        ratingAverages = contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['AggregatedRating'].notnull()][
            'AggregatedRating'].astype('int')
        C = ratingAverages.mean()
        m = ratingCounts.quantile(0.60)
        contentBasedRecommendedRecipes['Similarity'] = recipeSimilarity
        qualifiedRecipes2 = contentBasedRecommendedRecipes[
            (contentBasedRecommendedRecipes['Rating'] >= m) & (contentBasedRecommendedRecipes['Rating'].notnull()) & (
                contentBasedRecommendedRecipes['AggregatedRating'].notnull())]
        qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
        qualifiedRecipes2['AggregatedRating'] = qualifiedRecipes2['AggregatedRating'].astype('int')
        qualifiedRecipes2 = qualifiedRecipes2.sort_values(by='Similarity', ascending=False)
        qualifiedRecipes2 = qualifiedRecipes2.reset_index()
        return qualifiedRecipes2.head(12).to_json()

@app.route('/collaborative', methods=['POST'])
def getCollaborativeRecommendations():
    data = request.json
    if 'likedRecipeList' in data:
        likedRecipeList = data['likedRecipeList']
        return getCollaborativeRecommendations(likedRecipeList)  
    return pd.DataFrame().to_json()

def getCollaborativeRecommendations(recipe_ids):
    recommendItemLength = 4
    
    for recipe_id in recipe_ids:
        if recipe_id not in collaborativeMatrix.columns:
            # remove recipe that doesnt have more than {threshold} count of reviews from participating in correlation matching
            recipe_ids.remove(recipe_id)

    recommendRecipeIndices = []
    for recipe_id in recipe_ids:
        users_rating = collaborativeMatrix[recipe_id]
        similar_recipes = collaborativeMatrix.corrwith(users_rating, method='pearson')
        similar_recipes = pd.DataFrame(similar_recipes,columns=['correlation'])
        result_recipes = similar_recipes.join(df_rating['score']).sort_values(by='correlation', ascending=False)
        result_recipes = result_recipes[result_recipes['score'] > 4].sort_values(by = 'correlation', ascending = False) 
        recommendRecipeIndices.extend(result_recipes.iloc[1:recommendItemLength + 1].index.tolist())

    filtered_rows = pd.DataFrame(columns=(qualifiedRecipes.columns.tolist() + ['SourceRecipeId']))
    
    for k in range(len(recommendRecipeIndices)):
        for i in range(len(qualifiedRecipes)):
            if qualifiedRecipes.iloc[i]['RecipeId'] == recommendRecipeIndices[k]:
                filtered_rows = filtered_rows.append(qualifiedRecipes.iloc[i], ignore_index=True)
                filtered_rows.loc[filtered_rows.index[-1], 'SourceRecipeId'] = recipe_ids[math.floor(k / recommendItemLength)]
                break
    filtered_rows = filtered_rows.sort_values(by=['SourceRecipeId', 'score'], ascending=[True, False])
    print(filtered_rows)
    filtered_rows = filtered_rows.reset_index()
    returnValue = filtered_rows[['RecipeId', 'Name', 'Images', 'TotalTime', 'RecipeCategory', 'SourceRecipeId', 'score']]
    return returnValue.to_json()

#print(getCollaborativeRecommendations([56, 10744]))

@app.route('/knn', methods=['POST'])
def getKNNRecommendations():
    data = request.json
    if 'likedRecipeList' in data:
        likedRecipeList = data['likedRecipeList']
        return knn_recommender(likedRecipeList)  
    return pd.DataFrame().to_json()

def knn_recommender(recipe_ids, matrix = knnMatrix, n_recs = 10, n_result = 4):
    n_recs = n_result * len(recipe_ids)
    cf_knn_model= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
    cf_knn_model.fit(matrix)
    cf_recs = pd.DataFrame(columns=(qualifiedRecipes.columns.tolist() + ['SourceRecipeId', 'Distance']))
    
    for recipe_id in recipe_ids:
        recipe_index = process.extractOne(str(recipe_id),qualifiedRecipes['RecipeId'])[2]
        distances, indices = cf_knn_model.kneighbors(matrix.iloc[recipe_index,:].values.reshape(1,-1), n_neighbors=n_recs)
        recipe_rec_ids = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        iterNo = range(len(recipe_rec_ids))
        for i in iterNo:
            if i == n_result:
                break
            index_to_append = recipe_rec_ids[i][0]
            if not any(cf_recs['RecipeId'] == qualifiedRecipes.iloc[index_to_append]['RecipeId']):
                cf_recs = cf_recs.append(qualifiedRecipes.iloc[index_to_append], ignore_index=True)
                cf_recs.loc[cf_recs.index[-1], 'SourceRecipeId']  = recipe_id
                cf_recs.loc[cf_recs.index[-1], 'Distance']  = recipe_rec_ids[i][1]
            else:
                iterNo = iterNo + 1
    
    cf_recs = cf_recs[['RecipeId', 'Name', 'Images', 'TotalTime', 'RecipeCategory', 'SourceRecipeId', 'Distance']]
    return cf_recs.to_json()

#print(knn_recommender([56, 26499]))

@app.route('/datasetDump')
def getDatasetDump():
    return datasetDumpAlgolia.to_json()


if __name__ == "__main__":
    osPort = os.getenv("PORT")
    if osPort == None:
        port = 5000
    else:
        port = int(osPort)
    app.run(host='0.0.0.0', port=port)

# class ModelDataHolder:
#     def __init__(self, matrix)

C:\Users\CY\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.92:5000
Press CTRL+C to quit
127.0.0.1 - - [03/May/2024 21:38:10] "GET /popularity HTTP/1.1" 200 -
C:\Users\CY\AppData\Local\Temp\ipykernel_13164\1400386500.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
C:\Users\CY\AppData\Local\Temp\ipykernel_13164\1400386500.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualifiedR

   index  RecipeId                             Name  Rating  AggregatedRating  \
0   4533      9244                     Banana Bread      24                 5   
1   4475      9173                 Banana Nut Bread      39                 4   
2    643      1387    Perfectly Spiced Banana Bread      93                 5   
3   5987     11307           Moist Banana Nut Bread      42                 5   
4   2786      6627               Apple Banana Bread      73                 5   
5   1108      2885                  Banana Bread II      20                 5   
6   4594      9351          Sour Cream Banana Bread     607                 5   
7   4147      8754      Sweet Peanut Butter Cookies     121                 5   
8   5055     10131                     Banana Bread      54                 5   
9   4579      9327  Double Chocolate Banana Muffins     162                 5   

   Similarity  
0    0.619823  
1    0.374603  
2    0.362523  
3    0.357273  
4    0.338474  
5    0.32605

C:\Users\CY\AppData\Local\Temp\ipykernel_13164\1400386500.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
C:\Users\CY\AppData\Local\Temp\ipykernel_13164\1400386500.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualifiedRecipes2['AggregatedRating'] = qualifiedRecipes2['AggregatedRating'].astype('int')
127.0.0.1 - - [03/May/2024 21:52:07] "POST /content HTTP/1.1" 200 -


...

    index  RecipeId                                               Name  \
0     409       592                                        Fresh Salsa   
1    1074      2827                      Prize Winning Zucchini Relish   
2     610      1183                          Texas Style Picante Sauce   
3     657      1436                                        Chili Sauce   
4    6282     11672                                   Canning Tomatoes   
5     970      2675              Whole Wheat Pizza Dough & Pizza Sauce   
6    5865     11141                            All-Day Spaghetti Sauce   
7    5528     10732                           Homemade Spaghetti Sauce   
8    1864      4089                                    Spaghetti Sauce   
9    5870     11149                         Chili Con Carne With Beans   
10   1565      3594  Ground Beef Stuffed Green Bell Peppers II - Ov...   

    Rating  AggregatedRating  Similarity  
0        7                 4    0.328713  
1        5               

C:\Users\CY\AppData\Local\Temp\ipykernel_13164\1400386500.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
C:\Users\CY\AppData\Local\Temp\ipykernel_13164\1400386500.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualifiedRecipes2['AggregatedRating'] = qualifiedRecipes2['AggregatedRating'].astype('int')
127.0.0.1 - - [03/May/2024 21:53:48] "POST /content HTTP/1.1" 200 -


   index  RecipeId                           Name  Rating  AggregatedRating  \
0   4742      9545  Sour Cream Chicken Enchiladas      36                 5   
1   1557      3586         Chunky Chili Taco Soup      54                 5   
2   3989      8564          Chicken Tortilla Soup      23                 5   
3   6117     11470  Chicken Cordon Bleu over Rice      19                 5   
4   4867      9842             Crock Pot Bean Dip      26                 5   
5    428       619                  Egg Drop Soup      24                 5   
6   2138      4574          Chicken Tortilla Soup      58                 5   
7   6031     11367  Black Bean Tortilla Pinwheels      12                 5   
8    558       992        Jalapeno Pepper Poppers      12                 5   
9   4769      9713           Tortilla Dip Supreme      31                 5   

   Similarity  
0    0.277781  
1    0.230191  
2    0.213031  
3    0.212937  
4    0.203326  
5    0.202147  
6    0.187367  
7 

127.0.0.1 - - [03/May/2024 21:54:47] "GET /popularity HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 21:56:51] "OPTIONS /collaborative HTTP/1.1" 200 -
C:\Users\CY\AppData\Local\Temp\ipykernel_13164\1400386500.py:204: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_rows = filtered_rows.append(qualifiedRecipes.iloc[i], ignore_index=True)
C:\Users\CY\AppData\Local\Temp\ipykernel_13164\1400386500.py:204: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_rows = filtered_rows.append(qualifiedRecipes.iloc[i], ignore_index=True)
C:\Users\CY\AppData\Local\Temp\ipykernel_13164\1400386500.py:204: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_rows = filtered_rows.append(qualifiedRecipes.iloc[i], ignore_index=True)
C:\U

  RecipeId                                               Name RecipeCategory  \
1   113983                               Classic Saffron Rice     White Rice   
0    50004                                        Lil Smokies   Lunch/Snacks   
3   167914  The Best Fluffiest Buttermilk Pancakes on the ...      Breakfast   
2    10958                        Cheating Scalloped Potatoes   Lunch/Snacks   
5    69173   Kittencal's Italian Melt-In-Your-Mouth Meatballs           Meat   
4    63689      My Family's Favorite Sloppy Joes (Pizza Joes)   Lunch/Snacks   
7    32844            Pasta with Sausage, Tomatoes, and Cream  One Dish Meal   
6    28943                  Most Incredible No Fail Pie Crust            Pie   

  AggregatedRating Rating  ReviewCount  \
1                5     88         91.0   
0                5     66         69.0   
3                5     56         57.0   
2                5     53         54.0   
5                5   1023       1068.0   
4                5    726  